# Vectorisation

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ARCTraining/swd6_hpp/blob/master/docs/03_vectorisation.ipynb)

## [Broadcasting](https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html)

Broadcasting allows for operations with different shaped arrays.

It's implemented in many libraries, such as [NumPy](https://numpy.org/doc/stable/user/basics.broadcasting.html) and [xarray](https://xarray.pydata.org/en/v0.16.2/computation.html?highlight=Broadcasting#broadcasting-by-dimension-name).

![broadcasting.png](images/broadcasting.png)  

*[Image source](https://mathematica.stackexchange.com/questions/99171/how-to-implement-the-general-array-broadcasting-method-from-numpy)*

In [1]:
import numpy as np

In [2]:
nums_col = np.array([0, 10, 20, 30]).reshape(4, 1)
nums_col

array([[ 0],
       [10],
       [20],
       [30]])

In [3]:
nums_row = np.array([0, 1, 2])
nums_row

array([0, 1, 2])

In [4]:
nums_col + nums_row

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

In [5]:
import xarray as xr

In [6]:
nums_col = xr.DataArray([0, 10, 20, 30], [("col", [0, 10, 20, 30])])
nums_col

<xarray.DataArray (col: 4)> Size: 32B
array([ 0, 10, 20, 30])
Coordinates:
  * col      (col) int64 32B 0 10 20 30

In [7]:
nums_row = xr.DataArray([0, 1, 2], [("row", [0, 1, 2])])
nums_row

<xarray.DataArray (row: 3)> Size: 24B
array([0, 1, 2])
Coordinates:
  * row      (row) int64 24B 0 1 2

In [8]:
nums_col + nums_row

<xarray.DataArray (col: 4, row: 3)> Size: 96B
array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])
Coordinates:
  * col      (col) int64 32B 0 10 20 30
  * row      (row) int64 24B 0 1 2

## [Vectorisation](https://jakevdp.github.io/PythonDataScienceHandbook/02.03-computation-on-arrays-ufuncs.html)

Vectorisation allows the code to operate on multiple array elements at once, rather than looping through them one at a time.  

NumPy has many functions already vectorised for you, which have been optimised in C (i.e., they've been statically typed and compiled).

These are known as the universal functions ([ufuncs](https://numpy.org/doc/stable/reference/ufuncs.html)).

There are many operations available, for [maths](https://numpy.org/doc/stable/reference/ufuncs.html#math-operations) (e.g., `add` and `subtract`), [trigonometric](https://numpy.org/doc/stable/reference/ufuncs.html#trigonometric-functions) (e.g., `sin` and `cos`), [comparison](https://numpy.org/doc/stable/reference/ufuncs.html#comparison-functions) (e.g., `greater` and `less`), and [floating](https://numpy.org/doc/stable/reference/ufuncs.html#floating-functions) (e.g., `isnan`)

For example, instead of using `+`, you can use the equivalent ufunc `np.add`:

In [9]:
nums = np.arange(1_000_000)

In [10]:
%%timeit
[num + 2 for num in nums]

62.9 ms ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%%timeit
nums + 2  # adds 2 to every element by overloading the + (similar to broadcasting)

135 µs ± 3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
%%timeit
np.add(nums, 2)

146 µs ± 6.64 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


```{admonition} Question 1

If something doesn't vary for a given loop, should it be inside or outside of that loop?

```

## Create your own ufunc

You can vectorise any arbitrary Python function to a NumPy ufunc using [`np.vectorize`](https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html). Note that this is implemented using for loops, so doesn't necessarily offer a large performance advantage.

*Don't worry about what this function does, just focus on the vectorisation bit.*

In [13]:
import math

SQRT_2PI = np.float32((2.0 * math.pi) ** 0.5)

x = np.random.uniform(-3.0, 3.0, size=1_000_000)
mean = 0.0
sigma = 1.0


def my_function(x, mean, sigma):
    """Compute the value of a Gaussian probability density function at x with given mean and sigma."""
    return math.exp(-0.5 * ((x - mean) / sigma) ** 2.0) / (sigma * SQRT_2PI)

In [14]:
vectorized_function = np.vectorize(my_function)

In [15]:
%%timeit
vectorized_function(x, mean, sigma)

367 ms ± 4.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


```{admonition} Question 2

Can you run the unvectorised `my_function` directly on the same inputs (i.e., all of x)?

```

### Generalised universal functions ([gufuncs](https://numpy.org/doc/stable/reference/c-api/generalized-ufuncs.html))

Whereas ufuncs apply the function element-by-element, the generalized version (gufuncs) supports "sub-array" by "sub-array" operations. 

Numba has a nice implementation of these, which we will explore in the next lesson.

## Exercises

```{admonition} Exercise 1

What is broadcasting?

```

```{admonition} Exercise 2

What is vectorisation?

```

```{admonition} Exercise 3

How would you replace the `compute_reciprocals` function below with a vectorised version?

```

In [16]:
def compute_reciprocals(array):
    """
    Divides 1 by an array of values.
    """
    output = np.empty(len(array))
    for i in range(len(array)):
        output[i] = 1.0 / array[i]

    return output

In [17]:
big_array = np.random.randint(1, 100, size=1_000_000)
%timeit compute_reciprocals(big_array)

1.03 s ± 3.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


```{admonition} Exercise 4

Create your own vectorised ufunc that calculates the cube root of an array over all elements.

```

## {ref}`Solutions <vectorisation>`

## Key Points

```{important}

- [x] _Take advantage of broadcasting for different shaped arrays._
- [x] _Use vectorised functions where you can e.g., [NumPy ufuncs](https://numpy.org/doc/stable/reference/ufuncs.html)._

```

## Further information

### More information

- [Special ufuncs from SciPy](https://docs.scipy.org/doc/scipy/reference/special.html)

### Other options

- [Lazy loading](https://xarray.pydata.org/en/v0.16.2/dask.html) and [execution](https://tutorial.dask.org/01x_lazy.html)
    - Lazily loads metadata only, rather than eagerly loading data into memory.
    - Creates task graph of scheduled work awaiting execution (`.compute()`, more on that in the Parallelisation lesson).